<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [5]</a>'.</span>

# Georeferences videos

Input files are a video (\*.mp4) recorded using the camera app and a log file (\*.csv) created using the GPSLogger app.

Output file is a csv file containing frame number, UTC timestamp, latitude and longitude.

# Data 

* directory named like /home/aubrey/Desktop/roadside/jupyter notebooks/20200619
  * one or more video files named like 20200619_104803.mp4
  * one GPS log generated by GPSLogger app named like gpslogger_20200619.csv    

In [1]:
import cv2
import pandas as pd
import numpy as np
import exiftool
from datetime import datetime, timedelta
import pytz
import logging
import glob

In [2]:
# Default values for 3 parameters. May be changed by papermill.
DATAPATH = '/home/aubrey/Desktop/roadside/jupyter notebooks'
DATE = '20200619'
FRAME_INTERVAL = 1800

In [3]:
def get_video_start_stop(filename):
    """
    """  
    with exiftool.ExifTool() as et:
        createDate = et.get_tag('QuickTime:CreateDate', filename)
        logging.info(f'Exif Quicktime:CreateDate: {createDate} UTC')
        duration = et.get_tag('QuickTime:Duration', filename)
        logging.info(f'Exif QuickTime:Duration: {duration} seconds')
    createDate = datetime.strptime(createDate, '%Y:%m:%d %H:%M:%S')   
    start = createDate - timedelta(seconds=duration)
    logging.info(f'Video started at {start} UTC')
    stop = createDate
    logging.info(f'Video ended at {stop} UTC')
    return start, stop

#get_video_start_stop(filename)

In [4]:
def get_lat_lon(timestamp):
    timestamp = pd.to_datetime(timestamp)
    df = dfgps[dfgps.time==timestamp]
    if not df.empty:
        # There is a record for exact timestamp (unlikely); return lat lon.
        return dfgps.lat.values[0], dfgps.lon.values[0]
    else:
        # Estimate lat lon using linear interpolation records just prior and post timestamp 
        df1 = dfgps[dfgps.time<timestamp].tail(1)     
        df2 = dfgps[dfgps.time>timestamp].head(1)
        t1 = df1.time.values[0]
        t2 = df2.time.values[0]
        lat1 = df1.lat.values[0]
        lat2 = df2.lat.values[0]
        lon1 = df1.lon.values[0]
        lon2 = df2.lon.values[0] 
        fraction = (timestamp-t1)/(t2-t1)
        lat = lat1 + fraction*(lat2-lat1)
        lon = lon1 + fraction*(lon2-lon1)
        return lat, lon

#start, stop = get_video_start_stop(filename)    
#get_lat_lon(start)

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [5]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(funcName)s %(message)s",
    datefmt="%Y-%m-%dT%H:%M:%S%z",
    handlers=[
        logging.FileHandler("debug.log"),
        logging.StreamHandler(),
    ]
)

videolist = glob.glob(f'{DATAPATH}/{DATE}/*.mp4')
logging.info(f'videolist = {videolist}')
gpslogpath = glob.glob(f'{DATAPATH}/{DATE}/gpslogger_{DATE}.csv')[0]
logging.info(f'gpslogpath = {gpslogpath}')

for video in videolist:
    #videopath =  video
    frameinterval = FRAME_INTERVAL
    
    logging.info(f'Started processing {videopath}')
    logging.info(f'Reading GPS log from {gpslogpath}')
    dfgps = pd.read_csv(gpslogpath, parse_dates=['time'])
    dfgps['time'] = dfgps['time'].dt.tz_localize(None)

    logging.info('Building timestamp-location table.')
    start_time, stop_time = get_video_start_stop(videopath)
    data = list()
    cap = cv2.VideoCapture(videopath)
    i = 0
    while(cap.isOpened()):
        frame_exists, curr_frame = cap.read()
        if frame_exists:
            pos_msec = cap.get(cv2.CAP_PROP_POS_MSEC)
            timestamp = start_time + timedelta(milliseconds=pos_msec)
            lat, lon = get_lat_lon(timestamp)
            data.append([i, timestamp, lat, lon])
            i += 1
            if (i%frameinterval==0):
                logging.info(f'{i} {timestamp} {lat:7.4f} {lon:8.4f}')
        else:
            break
    cap.release()
    dfts = pd.DataFrame(data, columns=['frame', 'timestamp', 'lat', 'lon'])
    outputpath = videopath.replace('.mp4','_gps.csv')
    logging.info(f'Saving output to {outputpath}')                               
    dfts.to_csv(outputpath, index=False)
    logging.info(f'Finished processing {videopath}') 
logging.info('FINISHED ALL')    

2020-06-28T17:18:11+1000 [INFO] <module> videolist = ['/home/aubrey/Desktop/roadside/jupyter notebooks/20200619/20200619_104803.mp4']


2020-06-28T17:18:11+1000 [INFO] <module> gpslogpath = /home/aubrey/Desktop/roadside/jupyter notebooks/20200619/gpslogger_20200619.csv


NameError: name 'videopath' is not defined